In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
file_path = "../data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf"

print("문서로드 중")

loader = PyPDFLoader(file_path)
docs = loader.load()

문서로드 중


In [3]:
len(docs)

30

In [4]:
# page_content 가 비어있다 -> 이미지이기 때문에 text로 불러오면 아무것도 못가져옴
docs[0]

Document(metadata={'producer': 'ezPDF Maker', 'creator': 'ezPDF Maker', 'creationdate': '2025-10-12T19:22:25+09:00', 'moddate': '2025-10-12T19:22:25+09:00', 'source': '../data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf', 'total_pages': 30, 'page': 0, 'page_label': '1'}, page_content='')

In [ ]:
# uv pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
# nvcc -V
# uv pip install torch torchvision torchaudio --torch-backend=cu126

```
uv add easyocr
uv add -U PyMupdf
```

In [5]:
import fitz
import numpy as np
from PIL import Image
from langchain.docstore.document import Document

In [ ]:
# reader = easyocr.Reader(("ko", "en"))
# # reader = easyocr.Reader(("ko", "en"), gpu=True)
# file_path = "../data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf"
# with fitz.open(file_path) as pdf:
#     mat = fitz.Matrix(300, 300)
#     pix = pdf[0].get_pixmap(matrix=mat, alpha=False)
#     img = Image.frombytes("RGB")

In [8]:
import easyocr

reader = easyocr.Reader(("ko", "en"))

with fitz.open(file_path) as pdf:
    img = pdf[0]
    result = reader.readtext(img)
    result

ValueError: Invalid input type. Supporting format = string(file path or url), bytes, numpy array

In [6]:
def load_pdf_with_easyocr(
    pdf_path: str,
    languages=("ko","en"),   # 한국어+영어
    gpu=True,                # GPU 사용
    dpi=300,                 # OCR 품질용 렌더 DPI
    page_from=None,          # 1부터 시작, None=첫 페이지
    page_to=None             # 포함, None=마지막 페이지
):
    reader = easyocr.Reader(list(languages), gpu=gpu)
    docs = []

    with fitz.open(pdf_path) as pdf:
        total = len(pdf)
        start = 1 if page_from is None else max(1, page_from)
        end   = total if page_to is None else min(total, page_to)

        for p in range(start-1, end):
            page = pdf[p]
            zoom = dpi / 72.0
            mat = fitz.Matrix(zoom, zoom)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)

            arr = np.array(img)
            result = reader.readtext(arr, detail=0, paragraph=True)
            text = "\n".join(result) if result else ""

            docs.append(Document(
                page_content=text,
                metadata={"source": pdf_path, "page": p+1, "dpi": dpi, "ocr_engine": "EasyOCR"}
            ))

    return docs

In [9]:
docs = load_pdf_with_easyocr(file_path, languages=("ko", "en"), gpu=True, dpi=300)
docs

[Document(metadata={'source': '../data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf', 'page': 1, 'dpi': 300, 'ocr_engine': 'EasyOCR'}, page_content='SIMSUNG\n삼성전자 지속가능경영보고서 2025\n4 Journey Towards a Sustainable Future'),
 Document(metadata={'source': '../data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf', 'page': 2, 'dpi': 300, 'ocr_engine': 'EasyOCR'}, page_content='A Journey Towards a Sustainable Future 삼성전자 지속가능경영보고서 2025\nOur Company CEO 메시지 회사소개 기업 지배구조 중대성 평가 이해관계자 소통\nPlanet\nPeople 임직원 공급망 사회공헌 개인정보보호와 보안 제품 품질과 안전\n04 05 06\n[DX부문] 추진체계와 주요성과 기후변화 자원순환 수자원 오염물질\n[DS부문] 추진체계와 주요성과 기후변화 자원순환 수자원 오염물질\n35 45 57 53\n77\n27\n12\n22 27\n07\n76\n09\n78\n29\n55\n20\n32\nPrinciple 준법과 운리경영\nFacts & Figures 경제성과 사회성과 환경성과 사업부문별 환경성과\nAppendix 독립된 인종인의 인종보고서 76 Scope 1,2 온실가스 배출량 검증 의견서 77 Scope 3 온실가스 배출량 검증 의견서 78 GRI Index 80 TCFD 대조표 82 SASB 대조표 84 About This Report 86\n59\n62\n63\n68\n72\n삼성전자 지속가능경영보고서 2025\nOurCompany\nPlanet\nPeople

In [11]:
len(docs)

30

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunk = splitter.split_documents(docs)
len(chunk)

86